In [1]:
import pandas as pd
import numpy as np
import re 
import time


import requests as rq 
import bs4 as bs4 
import json
import tqdm
import glob

# Verificação

    Depois de processar os dados para cada vídeo único que temos, vou abrir o arquivo que criei anteriormente.

In [2]:
# ah mas tinhamos vistos que tinha menos chaves, mas existem paginas que tem mais chaves que essa, que aumentam nosso numero de colunas

df = pd.read_json("parsed_video_info.json", lines = True)
df.shape

(1811, 173)

In [3]:
# vou colocar pro pandas colocar todas as colunas
# quero ver todas para inspecionar manualmente as que me interessam.
# tem muitas colunas com infos repetidas
pd.set_option("display.max_columns",173)
df.head(1)

,content-alignment_watch-small,watch-playlist_player-height,watch-queue-header,watch-queue-info,watch-queue-info-icon,watch-queue-title,watch-queue-control-bar_control-bar-button,watch-queue-mole-info,watch-queue-control-bar-icon,watch-queue-icon_yt-sprite,watch-queue-title-container,watch-queue-count,watch-queue-menu_yt-uix-button-menu_yt-uix-button-menu-dark-overflow-action-menu_hid,watch-queue-menu-choice_overflow-menu-choice_yt-uix-button-menu-item,watch-queue-controls,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_prev-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-prev_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_play-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-play_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_pause-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_hid_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-pause_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_next-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-next_yt-sprite,watch-queue-items-container_yt-scrollbar-dark_yt-scrollbar,watch-queue-items-list,content-alignment_watch-player-playlist,watch-main-col,watch-title-container,watch-title,watch-secondary-actions_yt-uix-button-group,watch-view-count,watch-action-panels_yt-uix-button-panel_hid_yt-card_yt-card-has-padding,watch-time-text,watch-extras-section,watch-meta-item_yt-uix-expander-body,content_watch-info-tag-list,watch-sidebar,watch-playlist_player-height_hid,watch-sidebar-gutter_yt-card_yt-card-has-padding_yt-uix-expander_yt-uix-expander-collapsed,watch-sidebar-section,watch-sidebar-head,watch-sidebar-body,watch-sidebar-separation-line,watch-queue-mole,watch-queue,watch-queue-title-msg,watch-queue-count-msg,watch-queue-loading-template,watch7-container,watch7-main-container,watch7-main,watch7-preview,watch7-content,watch-header,watch7-headline,watch-headline-title,watch7-user-header,watch7-subscription-container,watch8-action-buttons,watch8-secondary-actions,watch8-sentiment-actions,watch7-views-info,watch-action-panels,watch-actions-share-loading,watch-actions-share-panel,watch-actions-transcript-loading,watch-actions-transcript,watch-transcript-container,watch-transcript-not-found,watch-actions-rental-required,watch-description,watch-description-content,watch-description-clip,watch-uploader-info,watch-description-text,watch-description-extras,watch-discussion,watch7-sidebar,watch7-sidebar-contents,watch7-sidebar-offer,watch7-sidebar-ads,watch7-sidebar-modules,watch-related,shared-addto-watch-later-login,og:site_name,og:url,og:title,og:image,og:image:width,og:image:height,og:description,al:ios:app_store_id,al:ios:app_name,al:ios:url,al:android:url,al:android:app_name,al:android:package,al:web:url,og:type,og:video:url,og:video:secure_url,og:video:type,og:video:width,og:video:height,og:video:tag,fb:app_id,channel_link_0,channel_link_1,channel_link_2,channel_link_3,watch-channel-brand-div,watch-channel-brand-div-text,watch-sidebar-live-chat,channel_link_4,channel_link_5,channel_link_6,channel_link_7,channel_link_8,channel_link_9,channel_link_10,channel_link_11,channel_link_12,watch-meta-item,watch-sidebar-discussion,yt-pl-watch-queue-overlay,watch-meta-item_has-image,channel_link_13,watch-skeleton,watch_history,watch_later,watch_related_mix,what_to_watch,ytd-watch-card-album-list-renderer,ytd-watch-card-collage-renderer,ytd-watch-card-single-image-renderer,ytd-watch-card-video-list-renderer,ytd-generic-watch-card,watch-card-header,watch-card-title,watch-card-labels,ytd-artist-watch-card-renderer,ytd-show-wa

In [4]:
# mario já preparou as colunas selecionadas e seguem abaixo, olhou cada uma das colunas e copiou aquelas que podem ser interessantes

colunas_selecionadas = ['watch-title', 'watch-view-count', 'watch-time-text', 'content_watch-info-tag-list', 'watch7-headline',
                    'watch7-user-header', 'watch8-sentiment-actions', "og:image", 'og:image:width', 'og:image:height',
                    "og:description", "og:video:width", 'og:video:height', "og:video:tag", 'channel_link_0']

In [5]:
# Aqui e um conjunto minimo de colunas, nao pegou todas que e minimanete achou que poderia ser interessantes
# mas sim parar criar um modelo inicial , temos titulo, n de views, data de publicação, info tag list, 
# headline que as vezes vem tag ou titulo, entao se a gente usar no modelo tem que limpar e entender
# user header e o dono do canal que fez o upolado do video, precisamos limpar essa coluna
# sentiment-action parece ser o numero de likes
# temos a imagem, caso resolva baixar as imagens e thumbnail pra poder usar como feature, extrair features das imagens
# formato do video, resolução, quase todas vai ser 1280x720, mas pode ser que um video de baixa resolução nao seja tao interessante
# temos ainda uma tag, que mario nao sabe ao certo, pode ser que sejam hashtags
# temos as mesmas colunas referentes a resolução, estão duplicadas
# a gente tem a descrição do video
# e temos o primeiro link que ele extraiu para canal, dentre o varios, nao vamos usar o link nesse momento mas se fossemos usar teriamos que verificar se os canais linkados ali sao os que fizeram uploads do video

df[colunas_selecionadas].head()

,watch-title,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
0,Data Science Interview for a Job,565 visualizações,Publicado em 5 de nov. de 2019,Pessoas e blogs,#datascience #datascientist #interview\n\n\n\n...,Przemek Chojecki\n\n\n\n\n\n\n\n\n\n\n\n\n\nCa...,565 visualizações\n\n\n\n\n\n\n\n21\n\nGostou ...,https://i.ytimg.com/vi/-26pXGRGXeI/maxresdefau...,1280.0,720.0,How to conduct a Data Science Job Interview? H...,1280.0,720.0,get a job,/channel/UCEq0oQsS-voRnSWbcviIDGA
1,Machine Learning Course A To Z || Beginner to ...,172.324 visualizações,Publicado em 10 de ago. de 2018,Educação,Machine Learning Course A To Z || Beginner to ...,Geek's Lesson\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,172.324 visualizações\n\n\n\n\n\n\n\n5.088\n\n...,https://i.ytimg.com/vi/-58kO_zYUGE/maxresdefau...,1280.0,720.0,Welcome to this free online class on machine l...,640.0,360.0,Ai and machine learning course,/channel/UCKXx22vOENUyHrVAADq7Z_g
2,Kaggle Live-Coding: RNNs for Sarcasm Detection...,1.361 visualizações,Transmitido ao vivo em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Live-Coding: RNNs for Sarcasm Detection...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,1.361 visualizações\n\n\n\n\n\n\n\n32\n\nGosto...,https://i.ytimg.com/vi/-9U84J178OQ/maxresdefau...,1280.0,720.0,Join Kaggle data scientist Rachael live as she...,1280.0,720.0,CS,/channel/UCSNeZleDn9c74yQc-EKnVTA
3,Can You Become a Data Scientist?,597.242 visualizações,Publicado em 14 de ago. de 2018,Educação,#data #science #datascientist\n\n\n\n Can Y...,365 Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\nCa...,597.242 visualizações\n\n\n\n\n\n\n\n9.039\n\n...,https://i.ytimg.com/vi/-AkBfBWr_Gw/maxresdefau...,1280.0,720.0,"So, you want to become a data scientist? Great...",1280.0,720.0,data scientist,/channel/UCEBpSZhI1X8WaP-kY_2LLcg
4,Machine Learning In 5 Minutes | Machine Learni...,102.368 visualizações,Publicado em 19 de fev. de 2019,Educação,#MachineLearning #MachineLearningAlgorithms #W...,Simplilearn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,102.368 visualizações\n\n\n\n\n\n\n\n345\n\nGo...,https://i.ytimg.com/vi/-DEL6SVRPw0/maxresdefau...,1280.0,720.0,This Machine Learning basics video will help y...,1280.0,720.0,simplilearn,/channel/UCsvqVGtbbyHaMoevxPAq9Fg


In [6]:
# salvar no formato feather, pq ele é flexivel pra se abrir em varias linguagem, e como e formato binario salva mais rapido
# quando estamos mais avançados na parte de processamento de dados, mario gosta de salvar em feather
# 
df[colunas_selecionadas].to_feather("raw_data.feather")

In [9]:
# aquele detalhe agora que nao temos as labels pra esses dados, nao tenho o que quero modelar que é o Y
# se eu vou gostar ou não do video, eu poderia baixar os historicos de minhas views mas nesse caso vamos ver como funciona o labeling
# como funciona a anotação das variaveis Y quando a gente precisa
# E como usar o active learning pra fazer essas anotações de forma mais eficiente com menos dinheiro e tempo
# como nosso projeto é simples vou pegar o df[colunas_selecionadas] e colocar em um csv 
# coloca-lo numa planilha do google sheet e colocar as labels que eu quero 

df[colunas_selecionadas].to_csv("raw_data_sem_lables.csv", index= False)

In [10]:
csv = pd.read_csv("raw_data_sem_lables.csv")
csv.head()

,watch-title,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
0,Data Science Interview for a Job,565 visualizações,Publicado em 5 de nov. de 2019,Pessoas e blogs,#datascience #datascientist #interview\n\n\n\n...,Przemek Chojecki\n\n\n\n\n\n\n\n\n\n\n\n\n\nCa...,565 visualizações\n\n\n\n\n\n\n\n21\n\nGostou ...,https://i.ytimg.com/vi/-26pXGRGXeI/maxresdefau...,1280.0,720.0,How to conduct a Data Science Job Interview? H...,1280.0,720.0,get a job,/channel/UCEq0oQsS-voRnSWbcviIDGA
1,Machine Learning Course A To Z || Beginner to ...,172.324 visualizações,Publicado em 10 de ago. de 2018,Educação,Machine Learning Course A To Z || Beginner to ...,Geek's Lesson\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,172.324 visualizações\n\n\n\n\n\n\n\n5.088\n\n...,https://i.ytimg.com/vi/-58kO_zYUGE/maxresdefau...,1280.0,720.0,Welcome to this free online class on machine l...,640.0,360.0,Ai and machine learning course,/channel/UCKXx22vOENUyHrVAADq7Z_g
2,Kaggle Live-Coding: RNNs for Sarcasm Detection...,1.361 visualizações,Transmitido ao vivo em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Live-Coding: RNNs for Sarcasm Detection...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,1.361 visualizações\n\n\n\n\n\n\n\n32\n\nGosto...,https://i.ytimg.com/vi/-9U84J178OQ/maxresdefau...,1280.0,720.0,Join Kaggle data scientist Rachael live as she...,1280.0,720.0,CS,/channel/UCSNeZleDn9c74yQc-EKnVTA
3,Can You Become a Data Scientist?,597.242 visualizações,Publicado em 14 de ago. de 2018,Educação,#data #science #datascientist\n\n\n\n Can Y...,365 Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\nCa...,597.242 visualizações\n\n\n\n\n\n\n\n9.039\n\n...,https://i.ytimg.com/vi/-AkBfBWr_Gw/maxresdefau...,1280.0,720.0,"So, you want to become a data scientist? Great...",1280.0,720.0,data scientist,/channel/UCEBpSZhI1X8WaP-kY_2LLcg
4,Machine Learning In 5 Minutes | Machine Learni...,102.368 visualizações,Publicado em 19 de fev. de 2019,Educação,#MachineLearning #MachineLearningAlgorithms #W...,Simplilearn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,102.368 visualizações\n\n\n\n\n\n\n\n345\n\nGo...,https://i.ytimg.com/vi/-DEL6SVRPw0/maxresdefau...,1280.0,720.0,This Machine Learning basics video will help y...,1280.0,720.0,simplilearn,/channel/UCsvqVGtbbyHaMoevxPAq9Fg
